In [1]:
import dxdata
import pandas as pd
import os

# Initialize dxdata engine
engine = dxdata.connect(dialect="hive+pyspark")

In [2]:
project = os.popen("dx env | grep project- | awk -F '\t' '{print $2}'").read().rstrip()
record = os.popen("dx describe *dataset | grep  record- | awk -F ' ' '{print $2}'").read().rstrip().split('\n')[0]
DATASET_ID = project + ":" + record
dataset = dxdata.load_dataset(id=DATASET_ID)

In [3]:
data = dataset['participant']


# Find by field name
field_eid = pheno.find_field(name="eid")

# Find by exact title
field_sex = pheno.find_field(title="Sex")
field_age = pheno.find_field(title="Age at recruitment")
field_height = pheno.find_field(title="Standing height | Instance 0")

In [4]:
field_list = [field_eid, field_height, field_sex, field_age]
field_list

[<Field "eid">, <Field "p50_i0">, <Field "p31">, <Field "p21022">]

## prepare recorders 

In [16]:
df_all_fields = pd.DataFrame(columns = ['entity','field_name','filed_title'])
for entity in dataset.entities:
    entity_name  = entity.name
    
    for field in entity.fields:
        field_name = field.name
        filed_title = field.title
        df_all_fields.loc[len(df_all_fields),] = [entity_name,field_name,filed_title]
df_all_fields['downloaded']=[False]*len(df_all_fields)
df_all_fields.to_csv('recorder_all_fields.csv',index=False)


In [24]:
for entity in df_all_fields['entity'].unique():
    df_target_entity = df_all_fields.loc[df_all_fields['entity']==entity,]
    df_target_entity.to_csv(f'recorder_{entity}.csv')

# data downloading codes 

In [ ]:
# 52,68 is missing 

In [18]:
fail_list,ind

([52, 68, 179, 815, 816, 820, 828], 931)

In [19]:
entity='participant'
ind = 931
interval = 100
bulk_num = 30
fail_list = [52, 68, 179, 815, 816, 820, 828]

In [ ]:
from datetime import datetime
start_time = datetime.now()
df_target_entity = pd.read_csv(f'recorder/recorder_{entity}.csv',index_col=0)
end = ind+interval

while ind < end:
    try:
        print(f"now download bulk {ind}, from {ind*bulk_num+1} to {ind*bulk_num+bulk_num}")
        field_names = df_target_entity.loc[ind*bulk_num+1:ind*bulk_num+bulk_num,'field_name']
        index = field_names.index
        field_lst = [data.find_field(x) for x in list(field_names)]

        df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
        df_download = df_download.toPandas()

        df_download.to_csv(f'data/{entity}/{ind}.csv',index=False)

        df_target_entity.loc[index, 'downloaded']=[True]*len(index)
        df_target_entity.loc[index, 'ind']=[ind]*len(index)
        df_target_entity.to_csv(f'recorder/recorder_{entity}.csv')
    except:
        print(f"error in bulk {ind}")
        fail_list+=[ind]
    ind+=1
print(f' starttime {start_time}, endtime {datetime.now()}')

now download bulk 931, from 27931 to 27960
now download bulk 932, from 27961 to 27990
now download bulk 933, from 27991 to 28020
now download bulk 934, from 28021 to 28050
now download bulk 935, from 28051 to 28080
now download bulk 936, from 28081 to 28110
now download bulk 937, from 28111 to 28140
now download bulk 938, from 28141 to 28170
now download bulk 939, from 28171 to 28200
now download bulk 940, from 28201 to 28230
now download bulk 941, from 28231 to 28260
now download bulk 942, from 28261 to 28290
now download bulk 943, from 28291 to 28320
now download bulk 944, from 28321 to 28350
now download bulk 945, from 28351 to 28380
now download bulk 946, from 28381 to 28410
now download bulk 947, from 28411 to 28440
now download bulk 948, from 28441 to 28470
now download bulk 949, from 28471 to 28500
now download bulk 950, from 28501 to 28530
now download bulk 951, from 28531 to 28560
now download bulk 952, from 28561 to 28590
now download bulk 953, from 28591 to 28620
now downloa

In [ ]:
%%bash 
for file in data/participant/*; do
    echo "$file"
    dx upload "$file" --path /"$file"
    rm "$file"
done

ind = 0
interval = 5

filenames = [f'data/{entity}/{x}.csv' for x in range(ind,ind+interval)]
df_compact = pd.concat(map(pd.read_csv, filenames))
df_compact.to_pickle(f'data/{entity}/{ind}-{interval-1}.pkl')
for file in filenames:
    os.remove(file)
